In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Upload DATA

In [ ]:
data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
data

# PREPROCESS DATA

In [ ]:
data["HomePlanet"].unique(), len(data["HomePlanet"].unique())

In [ ]:
data.dropna(inplace=True)
data

In [ ]:
data["Cabin"].unique(), len(data["Cabin"].unique())

In [ ]:
data["Cabin_Area"] = data["Cabin"].str.split("/").str[0]
data["Cabin_Area"]

In [ ]:
data["Cabin_Side"] = data["Cabin"].str.split("/").str[2]
data["Cabin_Side"]

In [ ]:
data["Destination"].unique(), len(data["Destination"].unique())

In [ ]:
data = pd.get_dummies(data, columns=["HomePlanet", "CryoSleep", "VIP", "Destination", "Cabin_Area", "Cabin_Side"])
data

In [ ]:
data = data.drop(columns=["Cabin", "Name", "PassengerId"])
data

In [ ]:
data = data[(data['Age'] >= 1) & (data['Age'] <= 100)]
data

In [ ]:
data["Age_segment"] = pd.cut(data["Age"], 3, labels=[1, 2, 3])
data["Age_segment"]

In [ ]:
data = data.drop(columns=["Age"])

In [ ]:
# Definir condiciones y categorías
conditions = [
    (data['RoomService'] >= 0) & (data['RoomService'] <= 200),
    (data['RoomService'] > 200) & (data['RoomService'] <= 1000),
    (data['RoomService'] > 1000) & (data['RoomService'] <= 3000),
    (data['RoomService'] > 3000) & (data['RoomService'] <= 10000)
]

categories = [1, 2, 3, 4]

# Aplicar np.select
data['RoomService_Segment'] = np.select(conditions, categories)
data["RoomService_Segment"]

In [ ]:
(data["RoomService_Segment"] == 3).sum()

In [ ]:
data = data.drop(columns=["RoomService"])
data

In [ ]:
# Definir condiciones y categorías
conditions = [
    (data["FoodCourt"] >= 0) & (data["FoodCourt"] <= 500),
    (data["FoodCourt"] > 500) & (data["FoodCourt"] <= 3000),
    (data["FoodCourt"] > 3000) & (data["FoodCourt"] <= 10000),
    (data["FoodCourt"] > 10000) & (data["FoodCourt"] <= 30000)
]

categories = [1, 2, 3, 4]

# Aplicar np.select
data["FoodCourt_Segment"] = np.select(conditions, categories)
data["FoodCourt_Segment"]

In [ ]:
(data["FoodCourt_Segment"] == 4).sum()

In [ ]:
data = data.drop(columns=["FoodCourt"])
data

In [ ]:
data["ShoppingMall"].describe()

In [ ]:
# Definir condiciones y categorías
conditions = [
    (data["ShoppingMall"] >= 0) & (data["ShoppingMall"] <= 50),
    (data["ShoppingMall"] > 50) & (data["ShoppingMall"] <= 300),
    (data["ShoppingMall"] > 300) & (data["ShoppingMall"] <= 1000),
    (data["ShoppingMall"] > 1000) & (data["ShoppingMall"] <= 15000)
]

categories = [1, 2, 3, 4]

# Aplicar np.select
data["ShoppingMall_Segment"] = np.select(conditions, categories)
data["ShoppingMall_Segment"]                 

In [ ]:
(data["ShoppingMall_Segment"] == 2).sum()

In [ ]:
data

In [ ]:
data = data.drop(columns=["ShoppingMall"])
data

In [ ]:
data["Spa"].describe()

In [ ]:
# Definir condiciones y categorías
conditions = [
    (data["Spa"] >= 0) & (data["Spa"] <= 200),
    (data["Spa"] > 200) & (data["Spa"] <= 1000),
    (data["Spa"] > 1000) & (data["Spa"] <= 7000),
    (data["Spa"] > 7000) & (data["Spa"] <= 25000)
]

categories = [1, 2, 3, 4]

# Aplicar np.select
data["Spa_Segment"] = np.select(conditions, categories)
data["Spa_Segment"]     

In [ ]:
data = data.drop(columns=["Spa"])
data

In [ ]:
data["VRDeck"].describe()

In [ ]:
# Definir condiciones y categorías
conditions = [
    (data["VRDeck"] >= 0) & (data["VRDeck"] <= 200),
    (data["VRDeck"] > 200) & (data["VRDeck"] <= 1000),
    (data["VRDeck"] > 1000) & (data["VRDeck"] <= 7000),
    (data["VRDeck"] > 7000) & (data["VRDeck"] <= 25000)
]

categories = [1, 2, 3, 4]

# Aplicar np.select
data["VRDeck_Segment"] = np.select(conditions, categories)
data["VRDeck_Segment"] 

In [ ]:
(data["VRDeck_Segment"] == 4).sum()

In [ ]:
data = data.drop(columns=["VRDeck"])
data

In [ ]:
for col in data.columns:
    if data[col].dtype == 'bool':
        data[col] = data[col].astype(int)
data

# MODEL

In [ ]:
import tensorflow as tf
tf.__version__

## DIVIDE THE DATA

In [ ]:
y = data["Transported"]
X = data.drop(columns=["Transported"])
X, y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), len(y_train), len(X_test), len(y_test)

## DO THE MODEL

In [ ]:
tf.random.set_seed(1)

model_0 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1)
])

model_0.compile(loss="binary_crossentropy",
               optimizer="Adam",
               metrics=["accuracy"])

history_0 = model_0.fit(X_train,
                        y_train,
                       epochs=10,
                       validation_data=(X_test, y_test)) 

### VISUALIZE THE CURVE

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    """
    Plot the training and validation loss and accuracy.

    Args:
    - history: Returned object from model.fit() in Keras.
    """
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))

    # Plotting loss
    axes[0].plot(history.history['loss'], label='Training Loss')
    axes[0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0].set_title('Loss')
    axes[0].set_ylabel('Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].legend()

    # Plotting accuracy
    axes[1].plot(history.history['accuracy'], label='Training Accuracy')
    axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[1].set_title('Accuracy')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history_0)

### IMPROVE THE MODEL

In [ ]:
tf.random.set_seed(1)

model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(1)
])

model_1.compile(loss="binary_crossentropy",
               optimizer="Adam",
               metrics=["accuracy"])

history_1 = model_1.fit(X_train,
                        y_train,
                        epochs=10,
                        validation_data=(X_test, y_test))

In [ ]:
plot_training_history(history_1)

In [ ]:
tf.random.set_seed(1)

model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(1)
])

model_2.compile(loss="binary_crossentropy",
               optimizer="Adam",
               metrics=["accuracy"])

history_2 = model_2.fit(X_train,
                        y_train,
                        epochs=100,
                        validation_data=(X_test, y_test))

In [ ]:
plot_training_history(history_2)

In [ ]:
loss, accuracy = model_2.evaluate(X_test, y_test)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
tf.random.set_seed(1)

model_3 = tf.keras.Sequential([
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(1)
])

model_3.compile(loss="binary_crossentropy",
               optimizer="Adam",
               metrics=["accuracy"])

history_3 = model_3.fit(X_train,
                        y_train,
                        epochs=30,
                        validation_data=(X_test, y_test),
                        verbose=1)

In [ ]:
plot_training_history(history_3)

In [ ]:
loss, accuracy = model_3.evaluate(X_test, y_test)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
tf.random.set_seed(4)

model_4 = tf.keras.Sequential([
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(1)
])

model_4.compile(loss="binary_crossentropy",
               optimizer="Adam",
               metrics=["accuracy"])

history_4 = model_4.fit(X_train,
                        y_train,
                        epochs=50,
                        validation_data=(X_test, y_test),
                        verbose=0)

In [ ]:
plot_training_history(history_4)

In [ ]:
loss, accuracy = model_4.evaluate(X_test, y_test)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
test_data_final = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test_data_final

# PREPROCESS DATA OF TEST CSV

In [ ]:
# test_data_final.dropna(inplace=True)

# Extracción de partes específicas de la columna "Cabin"
test_data_final["Cabin_Area"] = test_data_final["Cabin"].str.split("/").str[0]
test_data_final["Cabin_Side"] = test_data_final["Cabin"].str.split("/").str[2]

# Generación de variables dummies para las columnas especificadas
test_data_final = pd.get_dummies(test_data_final, columns=["HomePlanet", "CryoSleep", "VIP", "Destination", "Cabin_Area", "Cabin_Side"])

# Descartar columnas no necesarias y filtrar filas según la edad
test_data_final = test_data_final.drop(columns=["Cabin", "Name", "PassengerId"])
test_data_final = test_data_final[(test_data_final['Age'] >= 1) & (test_data_final['Age'] <= 100)]


# Reemplazar NaN con 20 en la columna "Age"
test_data_final["Age"].fillna(20, inplace=True)
# Segmentar la columna "Age"
test_data_final["Age_segment"] = pd.cut(test_data_final["Age"], 3, labels=[1, 2, 3])
test_data_final = test_data_final.drop(columns=["Age"])

# Definir condiciones y categorías para cada segmentación
def segment_column(data, column, bins, labels):
    conditions = [
        (data[column] >= bins[i]) & (data[column] <= bins[i+1])
        for i in range(len(bins) - 1)
    ]
    data[f"{column}_Segment"] = np.select(conditions, labels)
    data = data.drop(columns=[column])
    return data

# Segmentar las columnas
bins_values = {
    "RoomService": [0, 200, 1000, 3000, 10000],
    "FoodCourt": [0, 500, 3000, 10000, 30000],
    "ShoppingMall": [0, 50, 300, 1000, 15000],
    "Spa": [0, 200, 1000, 7000, 25000],
    "VRDeck": [0, 200, 1000, 7000, 25000]
}

labels_values = [1, 2, 3, 4]

for column, bins in bins_values.items():
    test_data_final = segment_column(test_data_final, column, bins, labels_values)

# Convertir valores booleanos a enteros
for col in test_data_final.columns:
    if test_data_final[col].dtype == 'bool':
        test_data_final[col] = test_data_final[col].astype(int)

test_data_final

In [ ]:
# Hacer predicciones con el modelo
predictions = model_3.predict(test_data_final)
predictions

In [ ]:
predictions_class = (predictions > 0.5).astype(int)
predictions_class

In [ ]:
(predictions_class == 1).sum()

In [ ]:
(y == 1).sum()

In [ ]:
# Agregar las predicciones al DataFrame
test_data_final['Transported'] = predictions_class
test_data_final

In [ ]:
submission = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission

In [ ]:
print(test_data_final.isna().sum())

In [ ]:
# Crear un nuevo DataFrame con PassengerId y Transported
final_submission = pd.concat([submission["PassengerId"], test_data_final['Transported']], axis=1)
final_submission.columns = ["PassengerId", "Transported"]

# Sustituir NaN por 0 en la columna "Transported"
final_submission['Transported'].fillna(0, inplace=True)

# Convertir 1s y 0s a True y False respectivamente
final_submission['Transported'] = final_submission['Transported'].astype(bool)

# Guardar el DataFrame en un archivo CSV
final_submission.to_csv("final_submission.csv", index=False)

# Mostrar el DataFrame
final_submission



In [ ]:
print(final_submission.isna().sum())